In [ ]:
! git clone https://github.com/ziye2chen/LLMs-for-Mathematical-Analysis.git

In [1]:
!pip install --upgrade pip
!pip install unsloth transformers trl datasets pandas matplotlib rich scipy sentencepiece

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----------- ---------------------------- 0.5/1.8 MB 764.3 kB/s eta 0:00:02
   ----------- ---------------------------- 0.5/1.8 MB 764.3 kB/s eta 0:00:02
   ----------------- ---------------------- 0.8/1.8 MB 745.8 kB/s eta 0:00:02
   ---------------------- ----------------- 1.0/1.8 MB 740.5 kB/s eta 0:00:02
   ---------------------- ----------------- 1.0/1.8 MB 740.5 kB/s e


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: To modify pip, please run the following command:
C:\Users\Admin\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


^C


  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
    --------------------------------------- 0.3/10.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/10.8 MB 1.7 MB/s eta 0:00:07
   -- ------------------------------------- 0.8/10.8 MB 1.1 MB/s eta 0:00:10
   --- ------------------------------------ 1.0/10.8 MB 1.2 MB/s eta 0:00:09
   ---- ----------------------------------- 1.3/10.8 MB 1.3 MB/s eta 0:00:08
   ----- ---------------------------------- 1.6/10.8 MB 1.2 MB/s eta 0:00:08
   ------- -------------------------------- 2.1/10.8 MB 1.4 MB/s eta 0:00:07
   -------- ------------------------------- 2.4/10.8 MB 1.4 MB/s eta 0:00:06
   --------- ------------------------------ 2.6/10.8 MB 1.4 MB/s eta 0:00:06
   ----------- ---------------------------- 3.1/10.8 MB 1.5 MB/s eta 0:00:06
   ------------- ------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
opentelemetry-proto 1.30.0 requires protobuf<6.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
import torch
print("Pytorch version：")
print(torch.__version__)
print("CUDA Version: ")
print(torch.version.cuda)
print("cuDNN version is :")
print(torch.backends.cudnn.version())

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 10240 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!d
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-Math-7B-bnb-4bit", # "unsloth/Meta-Llama-3.1-8B"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

# Classifier

In [ ]:
alpaca_prompt = """As a mathematical assistant, You need to analyze the problem to find out what type of problem it belongs to in Real Analysis. Provide the Problem_Type and the Knowledges which may be used to solve this problem.

### Problem:
{}

### Problem_Type:
{}

### Knowledge:
{}"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    Problem_Type    = examples["ProblemType"]
    Problem       = examples["Problem"]
    Knowledge  = examples["Solution"]
    texts = []
    for problem, problem_type, knowledge in zip(Problem, Problem_Type, Knowledge):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(problem, problem_type, knowledge) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset('csv',data_files = '/content/LLMs-for-Mathematical-Analysis/pretraining_data.csv', split='train')
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset, # dataset -> tokenized_train_dataset
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 2,

        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps = 5,
        max_steps = 60, # 300 -> 60 -> 10

        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
model.save_pretrained("MathAnalysis_Qwen_Classifier") # Local saving
tokenizer.save_pretrained("MathAnalysis_Qwen_Classifier")

In [ ]:
test_prompt = """As a mathematical assistant, You need to analyze the problem to find out what type of problem it belongs to in Real Analysis. Provide the Problem_Type and the Knowledges which may be used to solve this problem.

### Problem:
{}
"""

In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "MathAnalysis_Qwen_Classifier", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    test_prompt.format(
        "Prove that $n \sin(2n! e \pi)$ converges to $2\pi$ as $n \to \infty$.", # Problem
    )
], return_tensors = "pt").to("cuda")


In [ ]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask,
                   streamer = text_streamer, max_new_tokens = max_seq_length, pad_token_id = tokenizer.eos_token_id)

In [ ]:
# Tự tính max_new_tokens an toàn
input_len = inputs.input_ids.shape[-1]
max_total_length = max_seq_length  # ví dụ: 10240
safe_max_new_tokens = max_total_length - input_len

# In thử kiểm tra
print(f"Input tokens: {input_len}, Max new tokens: {safe_max_new_tokens}")

# Dùng TextStreamer để in ra từng dòng sinh token
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt=True)

# Sinh kết quả
_ = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    streamer=text_streamer,
    max_new_tokens=safe_max_new_tokens,
    pad_token_id=tokenizer.eos_token_id
)

# Problem Solver

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 10240 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!d
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-Math-7B-bnb-4bit", # "unsloth/Meta-Llama-3.1-8B"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
alpaca_prompt = """As a mathematical assistant, solve the following problem. Provide a detailed, step-by-step solution using rigorous mathematical reasoning. If the problem requires the use of the $\epsilon$-$\delta$ method (e.g., when proving limits or continuity), ensure that you apply it appropriately. Use precise mathematical language and notation throughout your solution.

### Problem_Type:
{}

### Problem:
{}

### Knowledge:
{}

### Solution:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    Problem_Type    = examples["ProblemType"]
    Problem       = examples["Problem"]
    Knowledge = examples["Solution"]
    Solution  = examples["Solution"]
    texts = []
    for problem_type, problem, knowledge, solution in zip(Problem_Type, Problem, Knowledge, Solution):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(problem_type, problem, knowledge, solution) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset('csv',data_files = '/content/LLMs-for-Mathematical-Analysis/pretraining_data.csv', split='train')
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
len(dataset)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset, # dataset -> tokenized_train_dataset
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 2,

        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps = 5,
        max_steps = 60, # 60 -> 10

        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
model.save_pretrained("MathAnalysis_Qwen_ProblemSolver") # Local saving
tokenizer.save_pretrained("MathAnalysis_Qwen_ProblemSolver")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "MathAnalysis_Qwen_ProblemSolver", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 10240,
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

alpaca_prompt = """As a mathematical assistant, solve the following problem. Provide a detailed, step-by-step solution using rigorous mathematical reasoning. If the problem requires the use of the $\epsilon$-$\delta$ method (e.g., when proving limits or continuity), ensure that you apply it appropriately. Use precise mathematical language and notation throughout your solution.

### Problem_Type:
{}

### Problem:
{}
"""

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Diﬀerentiation",
        "Define the piecewise linear function\n\\[\ng(x) =\n\\begin{cases}\nx & \\text{for } 0 \\leq x < 1/2, \\\\\n1 - x & \\text{for } 1/2 \\leq x < 1,\n\\end{cases}\n\\]\nand extend it to $\\mathbb{R}$ periodically. Show that\n\\[\nT(x) = \\sum_{n=0}^\\infty \\frac{1}{2^n} g(2^n x)\n\\]\nis continuous but nowhere differentiable."
    )
], return_tensors = "pt").to("cuda")


In [ ]:
# Tự tính max_new_tokens an toàn
input_len = inputs.input_ids.shape[-1]
max_total_length = 10240  # ví dụ: 10240
safe_max_new_tokens = max_total_length - input_len

# In thử kiểm tra
print(f"Input tokens: {input_len}, Max new tokens: {safe_max_new_tokens}")

# Dùng TextStreamer để in ra từng dòng sinh token
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt=True)

# Sinh kết quả
_ = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    streamer=text_streamer,
    max_new_tokens=safe_max_new_tokens,
    pad_token_id=tokenizer.eos_token_id
)